In [3]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pathlib import Path

repo_path = Path("/content/AudioSep")
if not repo_path.exists():
    !git clone https://github.com/Audio-AGI/AudioSep.git

%cd /content/AudioSep



/content/AudioSep


# Data preparation

In [ ]:
###sampling#####

import librosa
import numpy as np
import os
import soundfile as sf

# file path
audio_path = "/content/drive/MyDrive/4810_project/raw_data/AuMix_09_Jesus_tpt_vn.wav"
sep_1_path = "/content/drive/MyDrive/4810_project/raw_data/AuSep_1_tpt_09_Jesus.wav"
sep_2_path = "/content/drive/MyDrive/4810_project/raw_data/AuSep_2_vn_09_Jesus.wav"

# output path
output_dir = "/content/drive/MyDrive/4810_project/mix_audio_new/"
os.makedirs(output_dir, exist_ok=True)

# parameter
slice_duration = 5  # second
sample_rate = 32000  # sampling rate

# func
def slice_audio(audio, sample_rate, slice_duration):
    """将音频分割为固定长度的片段。"""
    slice_length = int(sample_rate * slice_duration)  # samples
    slices = [
        audio[i : i + slice_length]
        for i in range(0, len(audio), slice_length)
        if len(audio[i : i + slice_length]) == slice_length
    ]
    return slices

# 定义音频切片和保存函数
def process_and_save_slices(audio_path, output_folder, sample_rate, slice_duration):
    """save as..."""
    os.makedirs(output_folder, exist_ok=True)
    audio, sr = librosa.load(audio_path, sr=sample_rate)
    audio_slices = slice_audio(audio, sample_rate, slice_duration)

    # 保存切片到文件夹
    for i, slice_data in enumerate(audio_slices):
        output_file = os.path.join(output_folder, f"slice_{i+1}.wav")
        sf.write(output_file, slice_data, sample_rate)  # 保存音频
        print(f"Saved: {output_file}")
    print(f"Total slices saved: {len(audio_slices)}")

# apply
process_and_save_slices(
    audio_path,
    os.path.join(output_dir, "mixed_audio"),
    sample_rate,
    slice_duration,
)

process_and_save_slices(
    sep_1_path,
    os.path.join(output_dir, "instrument_1"),
    sample_rate,
    slice_duration,
)

process_and_save_slices(
    sep_2_path,
    os.path.join(output_dir, "instrument_2"),
    sample_rate,
    slice_duration,
)


Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_1.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_2.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_3.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_4.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_5.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_6.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_7.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_8.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_9.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_10.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_11.wav
Saved: /content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_12.wav
Saved: /content/drive/MyD

In [1]:
!pip install torchlibrosa==0.1.0 gradio==3.47.1 gdown lightning transformers==4.28.1 ftfy braceexpand webdataset soundfile wget h5py

In [4]:
#download pretrained model
checkpoints_dir = Path("checkpoint")
checkpoints_dir.mkdir(exist_ok=True)

models = (
    (
        "https://huggingface.co/spaces/badayvedat/AudioSep/resolve/main/checkpoint/audiosep_base_4M_steps.ckpt",
        checkpoints_dir / "audiosep_base_4M_steps.ckpt"
    ),
    (
        "https://huggingface.co/spaces/badayvedat/AudioSep/resolve/main/checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt",
        checkpoints_dir / "music_speech_audioset_epoch_15_esc_89.98.pt"
    )
)

for model_url, model_path in models:
    if not model_path.exists():
        !wget {model_url} -O {model_path}

# directly

In [ ]:
import torch
from pipeline import build_audiosep, separate_audio

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = build_audiosep(
      config_yaml='config/audiosep_base.yaml',
      checkpoint_path=str(models[0][1]),
      device=device)
audio_file = "/content/drive/MyDrive/4810_project/test/mixed_slice_5.wav"
text1 = 'trumpet'
output_file1='/content/drive/MyDrive/4810_project/test/sep_trumpet_slice_5_ori.wav'
text2 = 'violin'
output_file2='/content/drive/MyDrive/4810_project/test/sep_violin_slice_5_ori.wav'
# AudioSep processes the audio at 32 kHz sampling rate
separate_audio(model, audio_file, text1, output_file1, device)
separate_audio(model, audio_file, text2, output_file2, device)

# audio_file = 'exp31_water drops_mixture.wav'
# text = 'water drops'
# output_file='separated_audio.wav'

# # AudioSep processes the audio at 32 kHz sampling rate
# separate_audio(model, audio_file, text, output_file, device)

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommend

Loaded AudioSep model from [checkpoint/audiosep_base_4M_steps.ckpt]
Separating audio from [/content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_5.wav] with textual query: [trumpet]
Separated audio written to [/content/drive/MyDrive/4810_project/mix_audio_new/sep_trumpet_slice_5_ori.wav]
Separating audio from [/content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_5.wav] with textual query: [violin]
Separated audio written to [/content/drive/MyDrive/4810_project/mix_audio_new/sep_violin_slice_5_ori.wav]


#**tranfer learning**

In [ ]:
#load pretrained model
from pipeline import build_audiosep ,separate_audio
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model = build_audiosep(
    config_yaml='config/audiosep_base.yaml',
    checkpoint_path='checkpoint/audiosep_base_4M_steps.ckpt',
    device=device
)

# froze QueryNet
for param in pretrained_model.query_encoder.parameters():
    param.requires_grad = False

# change output_layer
pretrained_model.ss_model.output_layer = nn.Sequential(
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(1, 1)),
    nn.Conv2d(in_channels=512, out_channels=1, kernel_size=(1, 1))
)


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommend

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/content/AudioSep/models/CLAP/open_clip/factory.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.

Loaded AudioSep model from [checkpoint/audiosep_base_4M_steps.ckpt]


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchaudio
import os

# datapath
DATA = "/content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio"
DATA_target1= "/content/drive/MyDrive/4810_project/mix_audio_new/instrument_1"
DATA_target2 = "/content/drive/MyDrive/4810_project/mix_audio_new/instrument_2"
text_descriptions = ["trumpet", "violin"]


def generate_data_list(mixture_dir, target_dirs, descriptions):
    """
    Args:
        mixture_dir:
        target_dirs:
        descriptions:
    Returns:
        data_list: include (mixture_path, target_path, text)
    """
    data_list = []

    for target_dir, description in zip(target_dirs, descriptions):
        for mixture_file in os.listdir(mixture_dir):
            # 检查是否为混合音频文件
            if mixture_file.startswith("slice_") and mixture_file.endswith(".wav"):
                # 构造混合音频路径
                mixture_path = os.path.join(mixture_dir, mixture_file)

                # 构造目标音频路径
                target_file = mixture_file  # 假设目标文件与混合文件名一致
                target_path = os.path.join(target_dir, target_file)

                # 检查目标音频是否存在
                if os.path.exists(target_path):
                    data_list.append((mixture_path, target_path, description))
                else:
                    print(f"Target file not found: {target_path}")

    return data_list

# call data_list
data_list = generate_data_list(
    mixture_dir=DATA,
    target_dirs=[DATA_target1, DATA_target2],
    descriptions=text_descriptions
)

# print data_list
# print("Generated data_list:")
# for item in data_list:
#     print(item)


# 数据集类
class AudioDataset(Dataset):
    def __init__(self, data_list):
        """
        Args:
            data_list: include (mixture_path, target_path, text)
        """
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        mixture_path, target_path, text = self.data_list[idx]
        try:
            # 加载混合音频和目标音频
            mixture, sr_mixture = torchaudio.load(mixture_path)
            target, sr_target = torchaudio.load(target_path)

            # 确保采样率一致，当前数据为 32kHz，无需上采样
            if sr_mixture != 32000 or sr_target != 32000:
                raise ValueError(f"Sample rate mismatch: "
                                 f"{mixture_path} has {sr_mixture} Hz, "
                                 f"{target_path} has {sr_target} Hz.")
        except Exception as e:
            raise ValueError(f"Error loading audio files: {e}\n"
                             f"Mixture path: {mixture_path}\n"
                             f"Target path: {target_path}")

        return {'mixture': mixture, 'target': target, 'text': text}

# 创建 DataLoader
dataset = AudioDataset(data_list)
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

# 测试 DataLoader
# 当前数据的采样率为 32kHz
for batch in dataloader:
    print("Mixture shape:", batch['mixture'].shape)  # 打印混合音频张量形状
    print("Target shape:", batch['target'].shape)   # 打印目标音频张量形状
    print("Text:", batch['text'])           # 打印文本描述
    break

Mixture shape: torch.Size([4, 1, 160000])
Target shape: torch.Size([4, 1, 160000])
Text: ['trumpet', 'trumpet', 'trumpet', 'trumpet']


# Training

In [ ]:
#loss 优化器
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR

# L1 loss
loss_function = nn.L1Loss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义优化器
optimizer = optim.AdamW(pretrained_model.parameters(), lr=1e-4)

# 学习率调度
def lr_lambda_func(step):
    warmup_steps = 10000
    return min(1.0, step / warmup_steps)

scheduler = LambdaLR(optimizer, lr_lambda_func)
#训练权重
from torch import nn

# 迁移学习训练
num_epochs = 75
pretrained_model.train()

for epoch in range(num_epochs):
    epoch_loss = 0.0

    for batch in dataloader:
        optimizer.zero_grad()

        # 获取输入
        mixtures = batch['mixture'].to(device)
        texts = batch['text']
        targets = batch['target'].to(device)

        # 查询条件嵌入
        conditions = pretrained_model.query_encoder.get_query_embed(
            modality='text',
            text=texts,
            use_text_ratio=1.0
        )

        # 前向传播
        input_dict = {'mixture': mixtures, 'condition': conditions}
        output = pretrained_model.ss_model(input_dict)['waveform']



        # 计算损失
        loss = loss_function(output, targets)
        loss.backward()

        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")



Epoch [1/75], Loss: 0.0288
Epoch [2/75], Loss: 0.0280
Epoch [3/75], Loss: 0.0260
Epoch [4/75], Loss: 0.0258
Epoch [5/75], Loss: 0.0258
Epoch [6/75], Loss: 0.0244
Epoch [7/75], Loss: 0.0244
Epoch [8/75], Loss: 0.0232
Epoch [9/75], Loss: 0.0225
Epoch [10/75], Loss: 0.0225
Epoch [11/75], Loss: 0.0205
Epoch [12/75], Loss: 0.0201
Epoch [13/75], Loss: 0.0206
Epoch [14/75], Loss: 0.0198
Epoch [15/75], Loss: 0.0196
Epoch [16/75], Loss: 0.0195
Epoch [17/75], Loss: 0.0182
Epoch [18/75], Loss: 0.0184
Epoch [19/75], Loss: 0.0180
Epoch [20/75], Loss: 0.0174
Epoch [21/75], Loss: 0.0169
Epoch [22/75], Loss: 0.0165
Epoch [23/75], Loss: 0.0164
Epoch [24/75], Loss: 0.0160
Epoch [25/75], Loss: 0.0161
Epoch [26/75], Loss: 0.0160
Epoch [27/75], Loss: 0.0162
Epoch [28/75], Loss: 0.0160


KeyboardInterrupt: 

# save model

In [ ]:
# unfrozen query_encoder
for param in pretrained_model.query_encoder.parameters():
    param.requires_grad = True
print("Query encoder unfrozen!")
# for param in pretrained_model.query_encoder.parameters():
#     print(f"Requires grad: {param.requires_grad}")  # output: True
#save model
torch.save(pretrained_model.state_dict(), "checkpoint/audiosep_finetuned.pth")


Query encoder unfrozen!


In [ ]:
import torch
import pytorch_lightning as pl

class AudioSepLightningModel(pl.LightningModule):
    def __init__(self, pretrained_model):
        super(AudioSepLightningModel, self).__init__()
        self.model = pretrained_model

    def forward(self, x):
        return self.model(x)
# 定义模型实例
lightning_model = AudioSepLightningModel(pretrained_model)

# 加载普通权重
state_dict = torch.load("checkpoint/audiosep_finetuned.pth", map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
lightning_model.model.load_state_dict(state_dict)

# 验证模型是否加载成功
print("Model weights loaded into Lightning successfully!")


<ipython-input-17-2cc18fc1736f>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("checkpoint/audiosep_finetuned.pth", map_location=torch.device('cud

Model weights loaded into Lightning successfully!


# Test

In [ ]:
from pipeline import build_audiosep ,separate_audio
import torch
import torch.nn as nn
##test
audio_file = "/content/drive/MyDrive/4810_project/test/mixed_slice_5.wav"
text1 = 'trumpet'
output_file1='/content/drive/MyDrive/4810_project/test/sep_trumpet_slice_5_epch75.wav'
text2 = 'violin'
output_file2='/content/drive/MyDrive/4810_project/test/sep_violin_slice_5_epch75.wav'
# AudioSep processes the audio at 32 kHz sampling rate
separate_audio(lightning_model.model, audio_file, text1, output_file1, device)
separate_audio(lightning_model.model, audio_file, text2, output_file2, device)

Separating audio from [/content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_5.wav] with textual query: [trumpet]
Separated audio written to [/content/drive/MyDrive/4810_project/mix_audio_new/sep_trumpet_slice_5_epch50.wav]
Separating audio from [/content/drive/MyDrive/4810_project/mix_audio_new/mixed_audio/slice_5.wav] with textual query: [violin]
Separated audio written to [/content/drive/MyDrive/4810_project/mix_audio_new/sep_violin_slice_5_epch50.wav]


# Visualization and Spectrum Consistency Evaluation

In [ ]:
#spectral coherence
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

audio_files = {
    "Mixed Audio": '/content/drive/MyDrive/4810_project/test/mix_slice_5.wav',
    "target_violin": "/content/drive/MyDrive/4810_project/instrument_2/slice_5.wav",
    "target_trumpet": "/content/drive/MyDrive/4810_project/instrument_1/slice_5.wav",
     "sep_violin": "/content/drive/MyDrive/4810_project/test/sep_violin_slice_5_epch75.wav",
     "sep_trumpet": "/content/drive/MyDrive/4810_project/test/sep_trumpet_slice_5_epch75.wav",
    "trumpet":"/content/drive/MyDrive/4810_project/raw_data/AuSep_1_tpt_09_Jesus.wav",
    "violin":"/content/drive/MyDrive/4810_project/raw_data/AuSep_2_vn_09_Jesus.wav"
    # "sep_violin_original": "/content/drive/MyDrive/4810_project/test/sep_violin_slice_5_ori.wav",
    # "sep_trumpet_original": "/content/drive/MyDrive/4810_project/test/sep_trumpet_slice_5_ori.wav"
}

# 保存图片的目录
output_dir = "/content/drive/MyDrive/4810_project/visualizations/new/"
os.makedirs(output_dir, exist_ok=True)

# 用于存储频谱一致性结果
spectrum_consistency = {}

# 加载目标信号
target_violin_path = audio_files["target_violin"]
target_trumpet_path = audio_files["target_trumpet"]

target_violin, sr_violin = librosa.load(target_violin_path, sr=None)
target_trumpet, sr_trumpet = librosa.load(target_trumpet_path, sr=None)

for title, path in audio_files.items():
    # 加载音频文件
    y, sr = librosa.load(path, sr=None)

    # 绘制波形图并保存
    plt.figure(figsize=(14, 5))
    librosa.display.waveshow(y, sr=sr, alpha=0.8)
    plt.title(f'Waveform: {title}')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Amplitude')
    plt.grid()
    waveform_path = os.path.join(output_dir, f"{title}_waveform.png")
    plt.savefig(waveform_path, dpi=300)
    plt.close()

    # 计算频谱
    D = np.abs(librosa.stft(y))  # 短时傅里叶变换 (STFT)
    dB = librosa.amplitude_to_db(D, ref=np.max)

    # 绘制频谱图并保存
    plt.figure(figsize=(14, 6))
    librosa.display.specshow(dB, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
    plt.title(f'Spectrogram: {title}')
    plt.colorbar(format='%+2.0f dB')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Frequency (Hz)')
    spectrogram_path = os.path.join(output_dir, f"{title}_spectrogram.png")
    plt.savefig(spectrogram_path, dpi=300)
    plt.close()

    print(f"Saved {title} visualizations to:")
    print(f" - Waveform: {waveform_path}")
    print(f" - Spectrogram: {spectrogram_path}")

    # 计算频谱一致性（仅针对分离信号与目标信号）
    if "sep_violin" in title:
        separated_signal, sr = librosa.load(path, sr=None)
        assert sr == sr_violin, f"Sampling rates do not match for {title} and target violin!"

        # 对齐信号长度
        min_length = min(len(separated_signal), len(target_violin))
        separated_signal = separated_signal[:min_length]
        target_signal = target_violin[:min_length]

        # 计算频谱一致性
        target_stft = np.abs(librosa.stft(target_signal, n_fft=1024, hop_length=512))
        separated_stft = np.abs(librosa.stft(separated_signal, n_fft=1024, hop_length=512))

        # 频谱相关性
        correlation = np.corrcoef(target_stft.flatten(), separated_stft.flatten())[0, 1]
        # 均方误差
        mse = np.mean((target_stft - separated_stft) ** 2)

        # 保存结果
        spectrum_consistency["sep_violin"] = {
            "Spectrum Correlation": correlation,
            "Spectrum MSE": mse
        }

    elif "sep_trumpet" in title:
        separated_signal, sr = librosa.load(path, sr=None)
        assert sr == sr_trumpet, f"Sampling rates do not match for {title} and target trumpet!"

        # 对齐信号长度
        min_length = min(len(separated_signal), len(target_trumpet))
        separated_signal = separated_signal[:min_length]
        target_signal = target_trumpet[:min_length]

        # 计算频谱一致性
        target_stft = np.abs(librosa.stft(target_signal, n_fft=1024, hop_length=512))
        separated_stft = np.abs(librosa.stft(separated_signal, n_fft=1024, hop_length=512))

        # 频谱相关性
        correlation = np.corrcoef(target_stft.flatten(), separated_stft.flatten())[0, 1]
        # 均方误差
        mse = np.mean((target_stft - separated_stft) ** 2)

        # 保存结果
        spectrum_consistency["sep_trumpet"] = {
            "Spectrum Correlation": correlation,
            "Spectrum MSE": mse
        }

# 输出频谱一致性结果
print("\nSpectrum Consistency Results:")
for instrument, metrics in spectrum_consistency.items():
    print(f"{instrument}:")
    print(f"  Spectrum Correlation: {metrics['Spectrum Correlation']:.4f}")
    print(f"  Spectrum MSE: {metrics['Spectrum MSE']:.4f}")


Saved Mixed Audio visualizations to:
 - Waveform: /content/drive/MyDrive/4810_project/visualizations/new/Mixed Audio_waveform.png
 - Spectrogram: /content/drive/MyDrive/4810_project/visualizations/new/Mixed Audio_spectrogram.png
Saved target_violin visualizations to:
 - Waveform: /content/drive/MyDrive/4810_project/visualizations/new/target_violin_waveform.png
 - Spectrogram: /content/drive/MyDrive/4810_project/visualizations/new/target_violin_spectrogram.png
Saved target_trumpet visualizations to:
 - Waveform: /content/drive/MyDrive/4810_project/visualizations/new/target_trumpet_waveform.png
 - Spectrogram: /content/drive/MyDrive/4810_project/visualizations/new/target_trumpet_spectrogram.png
Saved sep_violin visualizations to:
 - Waveform: /content/drive/MyDrive/4810_project/visualizations/new/sep_violin_waveform.png
 - Spectrogram: /content/drive/MyDrive/4810_project/visualizations/new/sep_violin_spectrogram.png
Saved sep_trumpet visualizations to:
 - Waveform: /content/drive/MyDrive

In [ ]:
!pip install librosa mir_eval numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100699 sha256=fe6b77f7c1c08e50b1142b727aedcbd9499144aba89926f5ccec2f8039635745
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built mir_eval


# Audio Separation Performance Evaluation

In [ ]:
#Audio Separation Evaluation Metrics
import numpy as np
import librosa
import mir_eval

# 加载音频文件
def load_audio(file_path):
    return librosa.load(file_path, sr=None)[0]

# 示例输入文件路径
audio_file = "/content/drive/MyDrive/4810_project/test/mixed_slice_5.wav"
output_file1 = '/content/drive/MyDrive/4810_project/test/sep_trumpet_slice_5_ori.wav'
output_file2 = '/content/drive/MyDrive/4810_project/test/sep_violin_slice_5_ori.wav'
# output_file1 = '/content/drive/MyDrive/4810_project/test/sep_trumpet_slice_5_epch75.wav'
# output_file2 = '/content/drive/MyDrive/4810_project/test/sep_violin_slice_5_epch75.wav'
# 加载音频数据
mixed_signal = load_audio(audio_file)
separated_trumpet = load_audio(output_file1)
separated_violin = load_audio(output_file2)

# 目标信号路径
target_trumpet = "/content/drive/MyDrive/4810_project/instrument_1/slice_5.wav"
target_violin = "/content/drive/MyDrive/4810_project/instrument_2/slice_5.wav"

# 加载目标信号
reference_trumpet = load_audio(target_trumpet)
reference_violin = load_audio(target_violin)

import mir_eval
import numpy as np

# 初始化结果字典
results = {"SDR": {}, "SIR": {}, "SAR": {}, "SDRi": {}}

# 计算指标
for instrument, reference_signal in zip(["trumpet", "violin"], [reference_trumpet, reference_violin]):
    # 分离的信号
    if instrument == "trumpet":
        separated_signal = separated_trumpet
    elif instrument == "violin":
        separated_signal = separated_violin

    # 为 mir_eval 准备输入：目标信号和估计信号
    target = np.array([reference_signal])  # 目标信号
    estimated = np.array([separated_signal])  # 分离信号

    # 使用 mir_eval 计算指标
    sdr, sir, sar,_ = mir_eval.separation.bss_eval_sources(target, estimated)

    # 计算SDRi
    sdr_inverted = 10 * np.log10(np.sum(target ** 2) / np.sum((target - estimated) ** 2))  # 倒数计算SDRi

    # 存储结果
    results["SDR"][instrument] = sdr[0]
    # results["SIR"][instrument] = sir[0]
    # results["SAR"][instrument] = sar[0]
    results["SDRi"][instrument] = sdr_inverted  

# 打印结果
print("Separation Quality Metrics:")
for metric, values in results.items():
    print(f"{metric}:")
    for instrument, score in values.items():
        print(f"  {instrument}: {score:.2f} dB")


Separation Quality Metrics:
SDR:
  trumpet: -9.91 dB
  violin: 17.57 dB
SIR:
SAR:
SDRi:
  trumpet: -5.59 dB
  violin: 17.10 dB
